En este notebook voy a repetir lo que hay hecho en el notebook denominado ExpTransferenciaEntrenamiento 1 RegresionesLinealesAnalisisGenerico

Para eso uso el siguiente modelo:

$$Y(i,t) = \alpha_i + \beta \cdot t + \gamma_{inicial} \Theta_{inicial} + \gamma_{final} \Theta_{final}$$

Donde i representa el sujeto, t representa la sesion (aca se juntan una sesion del test, las doce de entrenamiento (separanto los niveles dentro de cada sesion) y el test final (esto habria que revisarlo si conviene o no).  

Como esta regresion lineal es un lio de definir, la idea es hacer un "stack" donde se transforme todo en un sistema de matrices con la forma:

$$ \bar{Y} = \bar{\beta} \cdot \bar{\bar{X}}$$ 

Donde si i va de 1 a 3 y t de 1 a 4 queda escrito de la forma :

$$ X = 
 \begin{pmatrix}
  \beta & \alpha_1 & \alpha_2 & \alpha_3 & \gamma_{inicial} & \gamma_{final} \\
  0 & 1 & 0 & 0 & 1 & 0\\
  1 & 1 & 0 & 0 & 0 & 0\\
  2 & 1 & 0 & 0 & 0 & 0\\
  3 & 1 & 0 & 0 & 0 & 1\\
  0 & 0 & 1 & 0 & 1 & 0\\
  1 & 0 & 1 & 0 & 0 & 0\\
  2 & 0 & 1 & 0 & 0 & 0\\
  3 & 0 & 1 & 0 & 0 & 1\\
  0 & 0 & 0 & 1 & 1 & 0\\
  1 & 0 & 0 & 1 & 0 & 0\\
  2 & 0 & 0 & 1 & 0 & 0\\
  3 & 0 & 0 & 1 & 0 & 1\\
 \end{pmatrix}
$$

Una vez generada esa matriz usamos el codigo que figura aca:

http://stackoverflow.com/questions/24393518/python-sklearn-linear-model-linearregression-working-weird

In [1]:
# Cargamos los scripts usuales
%matplotlib inline
%load_ext autoreload
%autoreload 2
#from scripts.db import *
#from scripts.plots import *
#from scripts.stats import *
from scripts.statsTransferencia import *
import numpy as np
import statsmodels.api as sm

In [19]:
#Cargamos los datos y los ordenamos en una dataframe (ojo que lo retoque del original para que ordene las columnas como corresponde)

import pandas as pd

resumen = dataNumerica()
data = pd.DataFrame.from_dict(resumen, orient='index')

#Trasnformamos los valores que interesan en columnas (Por ahora voy a hacer todo en angulos)
entrenamientoP = pd.DataFrame()
entrenamientoA = pd.DataFrame()
entrenamiento = pd.DataFrame()

entrenamientoP['TI'] = data[data['Orientacion'] == 'P30']['TestInicialAngulos'].map(lambda x: x[0])
entrenamientoA['TI'] = data[data['Orientacion'] == 'A30']['TestInicialAngulos'].map(lambda x: x[4])

# entrenamiento['TI'] = data[data['Orientacion'] != 'CONTROL']['TestInicialAngulos'].map(lambda x: x[0])
# entrenamiento['TI'] = data[data['Orientacion'] == 'A30']['TestInicialAngulos'].map(lambda x: x[4])
# entrenamiento['TF'] = data[data['Orientacion'] != 'CONTROL']['TestFinalAngulos'].map(lambda x: x[0])
# entrenamiento['TF'] = data[data['Orientacion'] == 'A30']['TestFinalAngulos'].map(lambda x: x[4])


orientaciones = {0:'EntrenamientoInicialEnAngulos',1:'EntrenamientoMedioEnAngulos',2:'EntrenamientoFinalEnAngulos'}

for i in range(4):
    for key, value in orientaciones.items():
        entrenamientoP['E'+str(key)+'D'+str(i)] = data[data['Orientacion'] == 'P30'][orientaciones[key]].map(lambda x: x[i])
        entrenamientoA['E'+str(key)+'D'+str(i)] = data[data['Orientacion'] == 'A30'][orientaciones[key]].map(lambda x: x[i])
        entrenamiento['E'+str(key)+'D'+str(i)] = data[data['Orientacion'] != 'CONTROL'][orientaciones[key]].map(lambda x: x[i])

entrenamientoP['TF'] = data[data['Orientacion'] == 'P30']['TestFinalAngulos'].map(lambda x: x[0])
entrenamientoA['TF'] = data[data['Orientacion'] == 'A30']['TestFinalAngulos'].map(lambda x: x[4])

/home/ionatan/Gits/IPyVisound/scripts/statsTransferencia.py:536: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame
  dataByAlias[cts.P_FaseActiva].replace('ExperimentoCompleto', 'TestFinal', inplace=True)
/home/ionatan/Gits/IPyVisound/scripts/statsTransferencia.py:508: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame
  dataByAlias[cts.P_OrientacionEntrenamiento].replace('A30', 'P30', inplace=True)
/home/ionatan/Gits/IPyVisound/scripts/statsTransferencia.py:510: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame
  dataByAlias[cts.P_OrientacionEntrenamiento].replace('P30', 'CONTROL', inplace=True)


Es horrible como procese los datos en el codigo original, pero bueno...

Confio en que la info en entrenamientoP y entrenamientoA es lo unico que necesito

In [17]:
resumen

{'Sujeto01': {'EntrenamientoFinal': [139, 164, 164, 152],
  'EntrenamientoFinalEnAngulos': [24.399999999999999,
   14.399999999999999,
   14.399999999999999,
   19.199999999999999],
  'EntrenamientoInicial': [36, 186, 170, 160],
  'EntrenamientoInicialEnAngulos': [65.599999999999994,
   5.6000000000000005,
   12.0,
   16.0],
  'EntrenamientoMedio': [101, 160, 152, 157],
  'EntrenamientoMedioEnAngulos': [39.600000000000001,
   16.0,
   19.199999999999999,
   17.199999999999999],
  'Evolucion': [46, 11, 38, 57, 117, 154, 5, 20],
  'EvolucionAngulos': [-29.899999999999995,
   -7.7999999999999989,
   -22.0,
   -30.699999999999999,
   -46.799999999999997,
   -61.600000000000009,
   -2.0,
   -8.0],
  'Orientacion': 'A30',
  'TestFinal': [131, 177, 170, 178, 118, 158, 6, 21],
  'TestFinalAngulos': [27.599999999999998,
   9.2000000000000011,
   12.0,
   8.8000000000000007,
   32.799999999999997,
   16.800000000000001,
   77.599999999999994,
   71.599999999999994],
  'TestInicial': [85, 166, 13